## **Amazon Neptune 데이터베이스**

Neptune은 AWS에서 제공하는 완전 관리형 그래프 데이터베이스입니다. 그래프 데이터베이스는 다양한 관계가 있는 고도로 연결된 데이터에 적합합니다. 많은 회사에서 다음 사용 사례에 그래프 데이터베이스를 사용합니다.

  - 추천 엔진: 그래프 데이터베이스를 사용하여 소셜 네트워크에서 사용자와 팔로워를 매핑하거나 전자 상거래 애플리케이션에서 고객을 항목 구매에 매핑할 수 있습니다. 유사한 사용자 또는 고객 간의 연결을 분석하여 팔로우할 친구 또는 추가 구매 항목에 대한 정확한 추천을 제공할 수 있습니다.

  - 사기 탐지: 결제 회사는 그래프 데이터베이스를 사용하여 사기 거래의 고리를 식별합니다. 이메일 주소, IP 주소 및 기타 공유 정보 간의 관계를 분석하여 의심스러운 활동에 플래그를 지정하는 것이 더 쉽습니다.
  
  - 지식 그래프: 그래프 데이터베이스를 사용하여 관련 정보를 연결하여 사람, 장소 및 개념 간의 연결을 표시할 수 있습니다. 이렇게 하면 상점 또는 지식 허브의 엔터티에 대한 풍부한 컨텍스트를 사용할 수 있습니다.

Neptune을 사용하면 완전히 관리되는 그래프 데이터베이스 환경을 얻을 수 있습니다. 즉, 인스턴스 장애 조치, 데이터베이스 백업 및 복구 또는 소프트웨어 업그레이드에 집중할 필요가 없습니다. 애플리케이션을 구축하고 고객에게 가치를 제공하는 데 집중할 수 있습니다.

**Workshop URL** : [Graph relationships with Amazon Neptune](https://aws.amazon.com/ko/getting-started/hands-on/purpose-built-databases/neptune/)
<br>
---

#### **1. Neptune 데이터베이스 생성**

이 모듈에서는 Neptune 데이터베이스를 생성합니다. 이 데이터베이스는 애플리케이션에서 **사기 탐지 서비스**를 강화하는 데 사용됩니다.

**위 Workshop URL에서 create a Neptune database 부분 참조**

* 생성 후 보안그룹은 Default VPC CIDR를 허용합니다. (Default VPC CIDR : 172.31.0.0/16)

---

#### **2. Neptune 데이터베이스 실습을 위한 환경 확인**

이 모듈에서는 Neptune 데이터베이스에 접속 테스트 및 실습 환경을 구성합니다.

---

* Neptune 데이터베이스 접속 정보 확인

In [ ]:
%graph_notebook_config

* 실습을 위한 파일 스크립트 다운로드

In [ ]:
!curl -sL https://s3.amazonaws.com/aws-data-labs/fraud-detection.tar | tar -xv

* 다음 명령을 실행하여 Neptune 클러스터 엔드포인트를 환경 변수로 설정합니다.

In [ ]:
%env NEPTUNE_ENDPOINT=fraud-detection-defaultvpc.cluster-cynsmct2kawd.ap-northeast-2.neptune.amazonaws.com

* 다음 명령을 실행하여 Neptune 데이터베이스에 대한 연결을 테스트합니다. 

In [ ]:
!python scripts/test_connection.py

In [ ]:
%%writefile scripts/test_connection.py
import os

from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

endpoint = os.environ["NEPTUNE_ENDPOINT"]

g = traversal().withRemote(
    DriverRemoteConnection(f"wss://{endpoint}:8182/gremlin", "g")
)

results = g.V().count().next()

print(f"Connected to Neptune! There are {results} vertices in the database")

#### **3. Neptune 그래프 데이터 모델 설계 및 샘플 데이터 로드**

이 모듈에서는 그래프 데이터베이스를 사용한 데이터 모델링의 기본을 배웁니다.<br>그런 다음 사기 탐지 서비스를 위한 데이터 모델을 설계하고 샘플 데이터로 데이터베이스를 로드합니다.

---

그래프 데이터베이스는 관계형 데이터베이스와 같이 과거에 사용했던 데이터베이스와 다를 수 있습니다.<br> 그래프 데이터베이스에 대해 알아야 할 몇 가지 주요 용어가 있습니다:


* Graph: 이것은 데이터베이스 전체를 나타냅니다. 다른 데이터베이스의 테이블과 유사합니다.
* Vertex: vertex(또는 node)은 그래프에서 항목을 나타냅니다. 일반적으로 사람, 장소, 용어와 같은 명사나 개념을 나타내는 데 사용됩니다.이 단원에서 사용되는 용어인 vertex의 복수형은 vertex입니다.
* Edge: 두 vertex(node) 사이의 연결입니다. Edge는 종종 엔티티 간의 관계를 나타냅니다. 예를 들어 함께 작업하는 두 사람은 WorksWith Edge로 연결될 수 있습니다.
* Label: 추가되는 vertext(node) 또는 edge의 유형을 나타내는 데 사용할 수 있습니다. 예를 들어, 애플리케이션의 사용자를 나타내는 사용자 label이 있는 vertex(node)와 사람들이 팔로우할 수 있는 관심사를 나타내는 관심사 label이 있는 vertex(node)이 있을 수 있습니다.
* Property: Property을 추가합니다: vertex(node)와 edge에 키-값 쌍을 추가할 수 있습니다. 이를 property라고 합니다. 예를 들어 사용자 vertex(node)에는 사용자 이름 property가 있습니다.

그래프를 쿼리할 때는 종종 한 vertex(node)에서 시작하여 가장자리를 가로지르며 원래 vertex(node)과의 관계를 찾습니다. <br>사기 탐지 사용 사례에서는 사용자 vertex(node)에서 시작하여 검토된 edge를 순회하여 사용자가 검토한 레스토랑 vertex(node)을 찾을 수 있습니다.

그래프 데이터 모델을 구축할 때는 애플리케이션의 엔티티와 이들이 서로 어떻게 연관되어 있는지 고려해야 합니다. <br>그래프 데이터베이스에서 모델링하는 정보는 기본 데이터 저장소에 저장하는 정보와 다를 수 있습니다.

예를 들어, 기본 데이터베이스에는 각 사용자의 생년월일에 대한 정보가 포함될 수 있습니다. 사용자의 생일에 특별 혜택을 보내는 등의 작업을 위해 기본 데이터베이스에서 이 정보를 사용할 수 있지만, 사기 탐지 서비스에서는 유용하지 않을 수 있습니다. <br>생년월일은 사기에 영향을 미칠 가능성이 거의 없으므로 해당 데이터베이스에서 완전히 제외할 수 있습니다.

반대로 기본 데이터베이스에 저장하지 않는 정보 중 그래프 데이터베이스에 유용할 수 있는 정보가 있을 수 있습니다. 사용자가 레스토랑에 대한 리뷰를 남길 때 리뷰를 남길 때 사용된 IP 주소는 저장하지 않을 수 있습니다. <br>하지만 동일한 IP 주소를 사용하는 봇의 사기 활동 클러스터를 찾는 사기 탐지 서비스에서는 IP 주소가 매우 유용할 수 있습니다.


이를 염두에 두고 사기 탐지 서비스의 데이터 모델에 대한 대략적인 예는 다음 다이어그램에 나와 있습니다.

<center> <img src="//d1.awsstatic.com/screenshots/databases-learn-page/neptune/nep-20.c3b8377912f7622165c760920ad81dc64d66b3c4.png" alt="Diagram of the fraud-detection service" title="Diagram of the fraud-detection service" class="cq-dd-image" width="50%" height="50%"> </center>

이전 데이터 모델에서 정점은 타원으로 표시됩니다. 세 가지 다른 유형의 이 예에는 네 개의 정점이 있습니다.

* User: 애플리케이션에서 사용자를 나타냅니다. 사용자 vertex에는 사용자 label 사용자 이름 property가 있습니다.
* Restaurant: 애플리케이션에서 레스토랑을 나타냅니다. 레스토랑 vertex 에는 레스토랑 label 과 이름 property가 있습니다.
* IPAddress: 사용자가 레스토랑을 리뷰할 때 사용한 IP address 를 나타냅니다. IPAddress vertex 에는 IPAddress label 과 주소 property가 있습니다.

또한 다이어그램에는 두 가지 유형의 세 가지 모서리가 있습니다.

* Reviewed: 사용자가 레스토랑에 대해 제출한 리뷰를 나타냅니다. Reviewed edge에는 Reviewed label과 평점 property이 있습니다.
* Used: 사용자가 리뷰에 IPAddress를 사용했음을 나타냅니다. Used edge에는 Used label이 있습니다.


이제 몇 가지 예제 데이터를 그래프 데이터베이스에 로드하여 액세스 패턴을 테스트합니다.

scripts/ 디렉터리에 bulk_load_database.py라는 파일이 있습니다. 파일 편집기에서 파일을 엽니다. 다음과 같은 내용이 표시됩니다.

이 파일은 두 개의 파일(scripts/vertices.json 및 scripts/edges.json)에서 데이터를 로드하고 데이터를 Neptune 데이터베이스에 삽입합니다. <br>코드를 살펴보고 애플리케이션 코드에서 정점과 에지를 만드는 방법을 확인하세요. <br>코드에서 정점과 에지를 추가하기 위해 Gremlin 패키지의 addV() 및 addE() 메서드를 사용합니다.

대량 로드 스크립트를 실행하고 데이터베이스에 레코드를 삽입하려면 터미널에서 다음 명령을 실행합니다.

In [ ]:
!python scripts/bulk_load_database.py

In [ ]:
%%writefile scripts/bulk_load_database.py
import json
import os

from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

endpoint = os.environ["NEPTUNE_ENDPOINT"]

g = traversal().withRemote(
    DriverRemoteConnection(f"wss://{endpoint}:8182/gremlin", "g")
)

with open("scripts/vertices.json", "r") as f:
    for row in f:
        data = json.loads(row)
        if data["label"] == "User":
            g.addV("User").property("username", data["username"]).next()
        elif data["label"] == "Restaurant":
            g.addV("Restaurant").property("name", data["name"]).next()
        elif data["label"] == "IPAddress":
            g.addV("IPAddress").property("address", data["address"]).next()

with open("scripts/edges.json", "r") as f:
    for row in f:
        data = json.loads(row)
        if data["label"] == "Used":
            g.V().has("User", "username", data["username"]).as_("user").V().has(
                "IPAddress", "address", data["ip_address"]
            ).as_("ip_address").addE("Used").from_("user").to("ip_address").next()
        elif data["label"] == "Reviewed":
            g.V().has("User", "username", data["username"]).as_("user").V().has(
                "Restaurant", "name", data["restaurant"]
            ).as_("restaurant").addE("Reviewed").from_("user").to(
                "restaurant"
            ).property(
                "rating", data["rating"]
            ).property(
                "username", data["username"]
            ).property(
                "restaurant", data["restaurant"]
            ).next()

print("Loaded data successfully!")


모든 레코드를 로드하는 데 잠시 시간이 걸립니다. 데이터가 성공적으로 로드되었음을 나타내는 출력이 표시됩니다.

로드 후 테이블의 정점 수를 확인하려면 scripts/test_connection.py 스크립트를 다시 실행합니다.

In [ ]:
!python scripts/test_connection.py 

In [ ]:
%%writefile scripts/test_connection.py 
import os

from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

endpoint = os.environ["NEPTUNE_ENDPOINT"]

g = traversal().withRemote(
    DriverRemoteConnection(f"wss://{endpoint}:8182/gremlin", "g")
)

results = g.V().count().next()

print(f"Connected to Neptune! There are {results} vertices in the database")


이 모듈에서는 그래프 데이터베이스로 작업하기 위한 기본 용어를 배웠습니다. <br>그런 다음 사기 탐지 서비스를 위한 데이터 모델을 설계했습니다. <br>마지막으로 몇 가지 샘플 데이터를 데이터베이스에 로드했습니다.

다음 모듈에서는 그래프 데이터베이스에 대해 몇 가지 쿼리를 실행하여 사기 활동을 식별하는 데 도움을 줍니다.

#### **4. 애플리케이션에서 그래프 데이터베이스 사용**

이 모듈에서는 애플리케이션에서 그래프 데이터베이스를 사용하는 방법을 배웁니다. 레스토랑 리뷰 애플리케이션에서 잠재적인 사기 활동의 클러스터를 식별하기 위해 몇 가지 그렘린 그래프 탐색 쿼리를 실행합니다.

---

그래프 데이터베이스는 애플리케이션 전반의 관계를 탐색하는 데 효율적입니다. 그래프 데이터베이스는 다른 데이터베이스로는 발견하기 어려운 개체 간의 연결을 찾는 데 사용할 수 있습니다. <br>이러한 이유로 사기 탐지 및 소셜 네트워킹 애플리케이션에서 자주 사용됩니다.

첫 번째 사용 사례부터 시작하겠습니다. 레스토랑에 별점 1점을 많이 남기는 자동화된 봇에 문제가 있다고 가정해 보겠습니다. <br>사용자들이 낮은 평점을 대량으로 남기면서 레스토랑의 평판을 훼손하고 있기 때문에 레스토랑 소유주는 서비스에 불만을 품고 있습니다. <br>봇 트래픽을 발견하고 문제가 되는 리뷰를 삭제하려고 합니다.

의심스러운 것으로 플래그가 지정된 IP 주소가 있다고 가정해 보겠습니다. 이 IP 주소를 사용한 사용자와 해당 사용자가 별점 1점 리뷰를 많이 남겼는지 확인하려고 합니다.

applications/ 디렉터리에 find_users_of_suspicious_ip_addresses.py라는 파일이 있습니다. 파일 편집기에서 해당 파일을 엽니다. 내용은 다음과 같아야 합니다.

In [ ]:
%%writefile application/find_users_of_suspicious_ip_addresses.py
import os

from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python import statics

statics.load_statics(globals())


endpoint = os.environ["NEPTUNE_ENDPOINT"]

g = traversal().withRemote(
    DriverRemoteConnection(f"wss://{endpoint}:8182/gremlin", "g")
)


def find_users_of_suspicious_ip_addresses(ip_address):
    results = (
        g.V()
        .has("IPAddress", "address", ip_address)
        .as_("ip_address")
        .in_("Used")
        .aggregate("ip_address_users")
        .outE("Reviewed")
        .has("rating", 1)
        .values("username")
        .groupCount()
        .order(local)
        .by(values, desc)
        .limit(local, 10)
        .toList()
    )

    return [
        {"username": k, "1-star reviews": v}
        for result in results
        for k, v in result.items()
    ]


suspicious_users = find_users_of_suspicious_ip_addresses("173.153.51.29")

for user in suspicious_users:
    print(
        f"User {user['username']} has written {user['1-star reviews']} one-star reviews."
    )


In [ ]:
!python application/find_users_of_suspicious_ip_addresses.py

사기일 가능성이 있는 사용자 두 명을 발견할 수 있었습니다.

이 접근 방식은 동일한 IP 주소를 사용하는 봇 사용자를 찾을 수 있지만, 공격자가 더 똑똑해져서 여러 IP 주소에 걸쳐 행동을 확산시킬 수 있습니다. <br>그래프 데이터베이스를 사용하여 손상된 IP 주소의 클러스터를 찾을 수도 있습니다.

application/ 디렉터리에 find_related_suspicious_ip_addresses.py라는 파일이 있습니다. 파일 편집기에서 이 파일을 엽니다. <br>내용은 다음과 같아야 합니다.

이 파일의 내용은 이전 파일과 유사합니다. 서비스에서 사용할 수 있는 함수와 유사한 find_related_suspicious_ip_addresses라는 함수가 있습니다. <br>이 함수는 IP 주소를 받아 의심스러운 IP 주소를 사용한 사용자가 사용한 모든 IP 주소를 반환합니다. <br> 이 함수는 악성 행위자 클러스터를 식별하는 데 사용될 수 있습니다.


In [ ]:
%%writefile application/find_related_suspicious_ip_addresses.py
import os

from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python import statics

statics.load_statics(globals())


endpoint = os.environ["NEPTUNE_ENDPOINT"]

g = traversal().withRemote(
    DriverRemoteConnection(f"wss://{endpoint}:8182/gremlin", "g")
)


def find_related_suspicious_ip_addresses(ip_address):
    results = (
        g.V()
        .has("IPAddress", "address", ip_address)
        .as_("ip_address")
        .in_("Used")
        .aggregate("ip_address_users")
        .out("Used")
        .where(neq("ip_address"))
        .values("address")
        .groupCount()
        .order(local)
        .by(values, desc)
        .limit(local, 10)
        .toList()
    )

    return [
        {"ip_address": k, "user_overlap": v}
        for result in results
        for k, v in result.items()
    ]


suspicious_ip_addresses = find_related_suspicious_ip_addresses("173.153.51.29")

for user in suspicious_ip_addresses:
    print(
        f"IP address {user['ip_address']} has {user['user_overlap']} overlapping users."
    )

In [ ]:
!python application/find_related_suspicious_ip_addresses.py

성공! 의심스러운 IP 주소를 하나 더 발견했습니다. <br> 이전 함수에서 이 IP 주소를 사용하여 해당 IP 주소에 의심스러운 사용자가 있는지 확인할 수 있습니다.

이 모듈에서는 애플리케이션에서 그래프 데이터베이스를 사용하는 방법을 살펴봤습니다. <br> 그렘린 쿼리 언어를 사용하여 그래프를 가로지르며 관련 엔티티를 찾고 사기성 활동을 식별했습니다.


---

#### **5. 시각적으로 다시 확인해보겠습니다.**

1. 전체 그래프 조회

In [ ]:
%%gremlin
g.V().outE().inV().path()

2. 전체 User 조회

In [ ]:
%%gremlin
g.V().hasLabel("User").outE().inV().path()

3. 전체 IPAddress 조회

In [ ]:
%%gremlin
g.V().hasLabel("IPAddress").inE().outV().path()

4. 전체 Restaurant 조회

In [ ]:
%%gremlin
g.V().hasLabel("Restaurant").inE().outV().path()

5. User를 리스트로 출력

In [ ]:
%%gremlin
g.V().hasLabel("User").values("username").toList()

6. 특정 유저(williamsantonio)에 대한 그래프를 출력

In [ ]:
%%gremlin
g.V().has("User", "username", "williamsantonio").outE().inV().path()

#### 이상 감지 - 1점 리뷰를 준 IP들

In [ ]:
%%gremlin
g.V().hasLabel("IPAddress")
  .where(__.in("Used").count().is(gt(1)))
  .as("ip_address")
  .in("Used")
  .outE("Reviewed")
  .has("rating", 1)
  .path().by(valueMap())

이상한 IP가 2개가 있습니다.
다시한번 IP기반으로 2번 이상이 상용된 IP를 조회해보겠습니다.

In [ ]:
%%gremlin
g.V().hasLabel("IPAddress")
  .where(__.in("Used").count().is(gt(1)))
  .as("ip_address")
  .in("Used")
  .outE("Reviewed")
  .has("rating", 1)
  .dedup()
  .path().by(valueMap())

173.153.51.29와 174.70.217.249 IP가 이상합니다.

In [ ]:
%%gremlin
g.V()
  .hasLabel("IPAddress")
  .has("address", within("173.153.51.29"))
  .as("ip_address")
  .in("Used")
  .aggregate("ip_address_users")
  .out("Used")
  .where(neq("ip_address"))
  .path().by(valueMap())

In [ ]:
%%gremlin
g.V()
  .hasLabel("IPAddress")
  .has("address", within("174.70.217.249"))
  .as("ip_address")
  .in("Used")
  .aggregate("ip_address_users")
  .out("Used")
  .where(neq("ip_address"))
  .path().by(valueMap())

### **범인은 "hhouston" 이었습니다.**

위에서 IP만 조회해봐도 1:1이 아닌 그래프가 존재합니다.